In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_miocardial()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.05)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (1700, 110)
5.62% missing data
Class distribution: (array([0, 1]), array([1429,  271]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 2),
                       (46, 9, 1),
                       (96, 9, 5),
                       (96, 11, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 2.58 sec - Loss 0.233828 - ACC 73.91% - ACC Mean 73.91% - AUC 82.18% - AUC Mean 82.18% - Deter 000
Ite 00050 - 0.97 sec - Loss 0.116764 - ACC 76.93% - ACC Mean 75.19% - AUC 82.22% - AUC Mean 81.68% - Deter 024
Ite 00076 - 0.82 sec - Loss 0.100943 - ACC 75.52% - ACC Mean 75.29% - AUC 82.27% - AUC Mean 81.70% - Deter 050
Early stop ite 76, rollback to correction of ite 26, whith acc of 75.89% and auc of 84.09%
OUR METHOD RUN 1/10 - acc: 75.89% +- 0.0% - auc: 84.09% +- 0.0%
Ite 00000 - 0.77 sec - Loss 0.239439 - ACC 75.70% - ACC Mean 75.70% - AUC 82.55% - AUC Mean 82.55% - Deter 000
Ite 00050 - 0.84 sec - Loss 0.126461 - ACC 75.48% - ACC Mean 74.96% - AUC 81.76% - AUC Mean 81.93% - Deter 016
Ite 00100 - 0.76 sec - Loss 0.098067 - ACC 76.61% - ACC Mean 75.98% - AUC 82.93% - AUC Mean 82.35% - Deter 023
Ite 00150 - 0.88 sec - Loss 0.083502 - ACC 75.82% - ACC Mean 75.42% - AUC 84.08% - AUC Mean 83.39% - Deter 017
Ite 00200 - 0.73 sec - Loss 0.073662 - ACC 75.72% - ACC Mean 76.23% 

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 74.01% +- 0.0% - auc: 80.15% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 56.91% +- 1.27% - auc: 62.12% +- 2.45%
SFIL RUN 2/10 - acc: 57.39% +- 4.45% - auc: 59.81% +- 4.04%
SFIL RUN 3/10 - acc: 56.75% +- 2.12% - auc: 58.95% +- 2.13%
SFIL RUN 4/10 - acc: 55.22% +- 2.26% - auc: 59.82% +- 3.51%
SFIL RUN 5/10 - acc: 56.91% +- 0.85% - auc: 60.51% +- 1.71%
SFIL RUN 6/10 - acc: 58.43% +- 2.3% - auc: 60.41% +- 2.3%
SFIL RUN 7/10 - acc: 57.33% +- 1.79% - auc: 58.63% +- 1.48%
SFIL RUN 8/10 - acc: 55.77% +- 1.33% - auc: 58.39% +- 1.55%
SFIL RUN 9/10 - acc: 56.9% +- 1.99% - auc: 61.32% +- 3.21%
SFIL RUN 10/10 - acc: 57.28% +- 1.45% - auc: 60.62% +- 1.95%
SFIL GLOBAL - acc: 56.89% +- 0.84% - auc: 60.06% +- 1.13%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 74.33% +- 2.68% - auc: 81.72% +- 1.46%
PFIL RUN 2/10 - acc: 74.56% +- 3.39% - auc: 82.19% +- 3.09%
PFIL RUN 3/10 - acc: 74.57% +- 3.53% - auc: 84.79% +- 1.23%
PFIL RUN 4/10 - acc: 72.51% +- 5.05% - auc: 79.46% +- 7.21%
PFIL RUN 5/10 - acc: 75.46% +- 3.66% - auc: 83.67% +- 3.57%
PFIL RUN 6/10 - acc: 71.74% +- 3.4% - auc: 77.12% +- 2.75%
PFIL RUN 7/10 - acc: 73.44% +- 4.34% - auc: 82.15% +- 4.51%
PFIL RUN 8/10 - acc: 73.28% +- 2.72% - auc: 81.97% +- 1.37%
PFIL RUN 9/10 - acc: 74.83% +- 2.88% - auc: 81.86% +- 4.03%
PFIL RUN 10/10 - acc: 73.55% +- 2.81% - auc: 80.94% +- 4.48%
PFIL GLOBAL - acc: 73.83% +- 1.08% - auc: 81.59% +- 2.01%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 62.58% +- 5.34% - auc: 68.94% +- 5.17%
SPOL RUN 2/10 - acc: 58.33% +- 4.4% - auc: 64.32% +- 2.71%
SPOL RUN 3/10 - acc: 63.45% +- 3.74% - auc: 67.91% +- 5.8%
SPOL RUN 4/10 - acc: 59.65% +- 2.87% - auc: 63.71% +- 5.75%
SPOL RUN 5/10 - acc: 59.13% +- 4.11% - auc: 64.8% +- 5.95%
SPOL RUN 6/10 - acc: 64.8% +- 8.71% - auc: 69.06% +- 11.15%
SPOL RUN 7/10 - acc: 59.07% +- 4.44% - auc: 65.54% +- 3.37%
SPOL RUN 8/10 - acc: 59.36% +- 3.04% - auc: 63.99% +- 4.31%
SPOL RUN 9/10 - acc: 59.24% +- 1.96% - auc: 62.73% +- 3.74%
SPOL RUN 10/10 - acc: 60.8% +- 3.31% - auc: 66.12% +- 5.19%
SPOL GLOBAL - acc: 60.64% +- 2.09% - auc: 65.71% +- 2.13%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 74.28% +- 2.94% - auc: 81.82% +- 3.74%
PPOL RUN 2/10 - acc: 72.33% +- 3.43% - auc: 79.57% +- 2.85%
PPOL RUN 3/10 - acc: 73.44% +- 2.33% - auc: 81.92% +- 1.97%
PPOL RUN 4/10 - acc: 73.32% +- 4.6% - auc: 83.34% +- 4.61%
PPOL RUN 5/10 - acc: 73.56% +- 4.49% - auc: 82.78% +- 3.21%
PPOL RUN 6/10 - acc: 73.09% +- 5.51% - auc: 80.79% +- 1.51%
PPOL RUN 7/10 - acc: 73.54% +- 4.14% - auc: 81.04% +- 4.44%
PPOL RUN 8/10 - acc: 73.02% +- 1.8% - auc: 80.47% +- 2.07%
PPOL RUN 9/10 - acc: 71.77% +- 2.9% - auc: 81.07% +- 2.36%
PPOL RUN 10/10 - acc: 73.73% +- 3.94% - auc: 79.95% +- 4.6%
PPOL GLOBAL - acc: 73.21% +- 0.68% - auc: 81.28% +- 1.13%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
